In [1]:
import pandas as pd
import numpy as np

file_path = r"A:\Seismic_data_analysis\Seismic-Data-analysis\Data\NGAsub_MegaFlatfile_RotD50_050_R211022_public.xlsx"
df = pd.read_excel(file_path)

df.head()

,NGAsubRSN,DatabaseRegion,NGAsubEQID,NGAsubSSN,Earthquake_Name,YEAR,MODY,HRMN,Earthquake_Magnitude,Hypocenter_Latitude_deg,...,T8pt500S,T9pt000S,T9pt500S,T10pt000S,T11pt000S,T12pt000S,T13pt000S,T14pt000S,T15pt000S,T20pt000S
0,1000001,Alaska,1000001,1000001,Aleutian_Isl-Alaska,2014,623,2053,7.96,51.6928,...,0.000082,0.000095,0.000103,0.000125,0.000138,0.000109,0.000089,0.000064,0.000053,0.000061
1,1000002,Alaska,1000001,1000002,Aleutian_Isl-Alaska,2014,623,2053,7.96,51.6928,...,0.000415,0.000395,0.000405,0.000390,0.000336,0.000332,0.000263,0.000215,0.000187,0.000119
2,1000003,Alaska,1000001,1000003,Aleutian_Isl-Alaska,2014,623,2053,7.96,51.6928,...,0.000092,0.000083,0.000086,0.000082,0.000091,0.000078,0.000071,0.000066,0.000060,0.000055
3,1000004,Alaska,1000001,1000004,Aleutian_Isl-Alaska,2014,623,2053,7.96,51.6928,...,0.000055,0.000053,0.000051,0.000055,0.000059,0.000058,0.000042,0.000035,0.000038,0.000028
4,1000005,Alaska,1000001,1000005,Aleutian_Isl-Alaska,2014,623,2053,7.96,51.6928,...,0.000129,0.000116,0.000128,0.000122,0.000130,0.000119,0.000096,0.000135,0.000162,0.000060


In [ ]:
df_fil = df[
    (df['Earthquake_Magnitude'] >= 4) & 
    (df['Rjb_km'] > 0) & 
    (df['Rjb_km'] <= 500) & 
    (df['Vs30_Selected_for_Analysis_m_s'] > 0) & 
    (df['Hypocenter_Depth_km'] > 0) &
    (df["Fault_Type"] >= 0)
    
].copy()

# Inputs
X = pd.DataFrame()
X['M']         = df_fil['Earthquake_Magnitude']
X['logVs']     = np.log10(df_fil['Vs30_Selected_for_Analysis_m_s'])
X['logRrup']   = np.log10(df_fil['Rjb_km'])
X['Hyp_depth'] = df_fil['Hypocenter_Depth_km']
X["Rup"] = df_fil["Rjb_km"]
X["Fault_Type"] = df_fil["Fault_Type"]
# Outputs
time_cols = [col for col in df_fil.columns if col.startswith('T') and col.endswith('S')]
output_cols = time_cols + ['PGA_g', 'PGV_cm_sec']

epsilon = 1e-8
y = np.log10(df_fil[output_cols].clip(lower=epsilon))

# Convert
X = X.to_numpy()
y = y.to_numpy()


print("Input shape:", X.shape)
print("Output shape:", y.shape)

Input shape: (54140, 6)
Output shape: (54140, 113)


In [ ]:
import torch
import torch.nn as nn

SEED = 42
torch.manual_seed(SEED)
np.random.seed(SEED)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
X = torch.tensor(X , dtype = torch.float32 , device = device )
y = torch.tensor(y , dtype = torch.float32 , device = device )

# test train split
perm = torch.randperm(X.shape[0] , device=device)

X , y = X[perm] , y[perm]

split = int(0.8 * (X.shape[0]))

X_train , X_test = X[ :split] , X[split : ]
y_train , y_test = y[ :split] , y[split : ]

# normalization 
X_mean = X_train.mean(dim=0)  
X_std  = X_train.std(dim=0)

X_train = (X_train - X_mean) / (X_std + 1e-8)  
X_test  = (X_test  - X_mean) / (X_std + 1e-8) 

y_mean = y_train.mean(dim=0)  
y_std  = y_train.std(dim=0)   

y_train = (y_train - y_mean) / (y_std + 1e-8)
y_test  = (y_test  - y_mean) / (y_std + 1e-8)

C:\Users\sanji\AppData\Local\Temp\ipykernel_8212\2967107730.py:9: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X = torch.tensor(X , dtype = torch.float32 , device = device )
C:\Users\sanji\AppData\Local\Temp\ipykernel_8212\2967107730.py:10: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y = torch.tensor(y , dtype = torch.float32 , device = device )


In [ ]:
# NN for each feature
def make_subnet(hidden=64, out_dim=32, ):
    return nn.Sequential(
        nn.Linear(1, hidden),       
        nn.LayerNorm(hidden),       
        nn.ReLU(),                  
               
        nn.Linear(hidden, hidden),
        nn.LayerNorm(hidden),
        nn.ReLU(),
        
        nn.Linear(hidden, out_dim), 
        nn.LayerNorm(out_dim),
    )

# Additive model architeture 
class NAM(nn.Module):
    def __init__(self, n_features=6, n_outputs=113, hidden=64, subnet_out=32, dropout=0.15):
        super().__init__()

        
        self.subnets = nn.ModuleList([
            make_subnet(hidden, subnet_out)
            for _ in range(n_features)
        ])


        self.output_layer = nn.Linear(n_features * subnet_out, n_outputs)

    def forward(self, x):
        # x shape: (batch_size, 6)
        outs = []
        for j, net in enumerate(self.subnets):
            xj = x[:, j].unsqueeze(1)  
            outs.append(net(xj))        
        concat = torch.cat(outs, dim=1)    
        return self.output_layer(concat) 

In [39]:
model = NAM(
    n_features=6,
    n_outputs=113,
    hidden=64,
    subnet_out=32,
).to(device)


optimizer = torch.optim.AdamW(model.parameters(), lr=3e-3, weight_decay=1e-4)
criterion = torch.nn.MSELoss()


scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(
    optimizer, T_0=50, T_mult=2, eta_min=1e-5
)

In [ ]:
def r2_score(y_true, y_pred):
    # y_true, y_pred: shape (N, 113)
    ss_res = ((y_true - y_pred) ** 2).sum(dim=0)              
    ss_tot = ((y_true - y_true.mean(dim=0)) ** 2).sum(dim=0)  
    
    r2_per_output = 1.0 - ss_res / (ss_tot + 1e-8)          
    r2_per_output = r2_per_output.clamp(min=-1.0)           
    
    return r2_per_output.mean().item(), r2_per_output        

In [ ]:
EPOCHS     = 300
BATCH_SIZE = 1024
PATIENCE   = 30  

N_train = X_train.shape[0]


best_r2         = -float('inf')
best_state      = None
epochs_no_improve = 0

for epoch in range(EPOCHS):

    #shuffle
    perm = torch.randperm(N_train, device=device)
    X_tr_shuf = X_train[perm]
    y_tr_shuf = y_train[perm]


    model.train()
    total_loss = 0.0
    n_batches  = 0

    for start in range(0, N_train, BATCH_SIZE):
        end     = start + BATCH_SIZE
        X_batch = X_tr_shuf[start:end]   # shape (1024, 6)
        y_batch = y_tr_shuf[start:end]   # shape (1024, 113)

        optimizer.zero_grad()          
        pred = model(X_batch)           
        loss = ((pred - y_batch) ** 2).mean()   
        loss.backward()                 

        
        nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

        optimizer.step()                 # update weights
        total_loss += loss.item()
        n_batches  += 1

    scheduler.step(epoch)   # update learning rate

   
    model.eval() 
    with torch.no_grad():
        val_pred = model(X_test)     
        mean_r2, per_r2 = r2_score(y_test, val_pred)


    if mean_r2 > best_r2 + 1e-5:
        best_r2    = mean_r2

    if epoch % 10 == 0:
        neg_outputs = (per_r2 < 0).sum().item()
        print(f"Epoch {epoch:4d} | loss {total_loss/n_batches:.4f} | "
              f"val R² {mean_r2:.4f} | outputs with R²<0: {neg_outputs}")


model.load_state_dict({k: v.to(device) for k, v in best_state.items()})

Epoch    0 | loss 0.3341 | val R² 0.6459 | outputs with R²<0: 0
Epoch   10 | loss 0.3256 | val R² 0.6575 | outputs with R²<0: 0
Epoch   20 | loss 0.3203 | val R² 0.6573 | outputs with R²<0: 0
Epoch   30 | loss 0.3176 | val R² 0.6611 | outputs with R²<0: 0
Epoch   40 | loss 0.3155 | val R² 0.6621 | outputs with R²<0: 0
Epoch   50 | loss 0.3144 | val R² 0.6629 | outputs with R²<0: 0
Epoch   60 | loss 0.3210 | val R² 0.6591 | outputs with R²<0: 0
Epoch   70 | loss 0.3225 | val R² 0.6598 | outputs with R²<0: 0
Epoch   80 | loss 0.3204 | val R² 0.6569 | outputs with R²<0: 0
Epoch   90 | loss 0.3231 | val R² 0.6611 | outputs with R²<0: 0
Epoch  100 | loss 0.3205 | val R² 0.6608 | outputs with R²<0: 0
Epoch  110 | loss 0.3143 | val R² 0.6635 | outputs with R²<0: 0
Epoch  120 | loss 0.3132 | val R² 0.6642 | outputs with R²<0: 0
Epoch  130 | loss 0.3137 | val R² 0.6661 | outputs with R²<0: 0
Epoch  140 | loss 0.3108 | val R² 0.6665 | outputs with R²<0: 0
Epoch  150 | loss 0.3099 | val R² 0.6667

<All keys matched successfully>

In [ ]:
model.eval()
with torch.no_grad():
    final_pred = model(X_test)
    mean_r2, per_r2 = r2_score(y_test, final_pred)

per_r2_np = per_r2.cpu().numpy()

print(f"\nFinal mean R²:           {mean_r2:.4f}")
print(f"Median R² across outputs: {per_r2_np.mean():.4f}")  
print(f"Worst output R²:          {per_r2_np.min():.4f}")
print(f"Best  output R²:          {per_r2_np.max():.4f}")
print(f"Outputs with R² > 0.5:    {(per_r2_np > 0.5).sum()}/{len(per_r2_np)}")
print(f"Outputs with R² < 0:      {(per_r2_np < 0.0).sum()}/{len(per_r2_np)}")


Final mean R²:           0.6708
Median R² across outputs: 0.6708
Worst output R²:          0.5828
Best  output R²:          0.8207
Outputs with R² > 0.5:    113/113
Outputs with R² < 0:      0/113
